In [11]:
import threading
import time

# Shared database simulation
database = {"account_A": 0}

# Function for transaction TX
def transaction_TX(amount):
    """
    Transaction TX: Reads, modifies, writes, and then rolls back the balance of account A.
    """
    print("\n[TX] Starts")
    time.sleep(1)  # TX starts at t1
    local_balance = database["account_A"]
    print(f"[TX - t1] Reads balance: ${local_balance}")

    # Perform the addition
    time.sleep(1)  # TX operates at t2
    local_balance += amount
    print(f"[TX - t2] Adds ${amount}, new local balance: ${local_balance}")

    # TX writes the updated value at t3
    time.sleep(1)
    database["account_A"] = local_balance
    print(f"[TX - t3] Writes updated balance locally(not committed): ${database['account_A']}")

    # Simulate rollback at t5
    time.sleep(2)  # Allow TY to read before rollback
    database["account_A"] = database["initial_balance"]  # Rollback to initial value
    print(f"[TX - t5] Rollback: Reverting to initial balance: ${database['account_A']}")

# Function for transaction TY
def transaction_TY():
    """
    Transaction TY: Reads the balance after TX modifies it but before TX rolls back.
    """
    time.sleep(3)  # TY starts at t4, after TX writes the updated balance
    print("\n[TY] Starts")
    local_balance = database["account_A"]
    print(f"[TY - t4] Reads balance: ${local_balance}")

    # TY commits the dirty read
    time.sleep(1)  # TY commits at t6
    database["account_A"] = local_balance
    print(f"[TY - t6] Commits balance: ${database['account_A']}")

# Main simulation function
def dirty_read_simulation():
    """
    Simulates the Dirty Read Problem with user inputs for the initial balance and TX operation.
    """
    print("\n=== Dirty Read Problem Simulation ===")
    initial_balance = int(input("Enter the initial balance of account A: "))
    database["account_A"] = initial_balance
    database["initial_balance"] = initial_balance  # Store initial balance for rollback

    # TX configuration
    print("\n[TX Configuration]")
    tx_amount = int(input("Enter the amount to add by TX: "))

    # Start both transactions in threads
    tx_thread = threading.Thread(target=transaction_TX, args=(tx_amount,))
    ty_thread = threading.Thread(target=transaction_TY)

    tx_thread.start()
    ty_thread.start()

    # Wait for both threads to finish
    tx_thread.join()
    ty_thread.join()

    # Final output and explanation
    print("\nFinal balance after simulation:", database["account_A"])
    print(
        "\n=== Explanation of Dirty Read Problem ===\n"
        f"At time t1, Transaction TX reads the balance of account A, which is ${initial_balance}.\n"
        f"At time t2, TX adds ${tx_amount}, so the new local balance becomes ${initial_balance + tx_amount}.\n"
        f"At time t3, TX writes back the updated balance ${initial_balance + tx_amount}.\n"
        f"At time t4, Transaction TY reads the uncommitted balance ${initial_balance + tx_amount} (dirty read).\n"
        f"At time t5, TX rolls back, reverting the balance to ${initial_balance}.\n"
        "However, TY commits the dirty read value at time t6, leading to incorrect and inconsistent data.\n"
        "This demonstrates the Dirty Read Problem: a transaction (TY) reads uncommitted changes from another transaction (TX), "
        "which may later be rolled back, causing integrity issues."
    )

# Run the simulation
dirty_read_simulation()



=== Dirty Read Problem Simulation ===
Enter the initial balance of account A: 300

[TX Configuration]
Enter the amount to add by TX: 50

[TX] Starts
[TX - t1] Reads balance: $300
[TX - t2] Adds $50, new local balance: $350
[TX - t3] Writes updated balance locally(not committed): $350
[TY] Starts
[TY - t4] Reads balance: $350

[TY - t6] Commits balance: $350
[TX - t5] Rollback: Reverting to initial balance: $300

Final balance after simulation: 300

=== Explanation of Dirty Read Problem ===
At time t1, Transaction TX reads the balance of account A, which is $300.
At time t2, TX adds $50, so the new local balance becomes $350.
At time t3, TX writes back the updated balance $350.
At time t4, Transaction TY reads the uncommitted balance $350 (dirty read).
At time t5, TX rolls back, reverting the balance to $300.
However, TY commits the dirty read value at time t6, leading to incorrect and inconsistent data.
This demonstrates the Dirty Read Problem: a transaction (TY) reads uncommitted cha

In [ ]:
a